In [ ]:
import pandas as pd
#내가 뽑아낼 죄표 구하기 
# data = pd.read_csv('../preprocessing_data/최종/living_alone.csv')
data = pd.read_csv('../preprocessing_data/최종/union_area.csv', encoding='cp949')

data.head()


#크롤링 할때 좌표값도 같이 넣어주기 

In [13]:
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import time
from collections import defaultdict
import random
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm

def circle_data_crawling(circle_id, circle_num,where_x, where_y, school, mart):
    lst = []
    try:
        #아이디가 없는것도 있다 그런거는 스킵
        driver.find_element(By.XPATH,f'//*[@id="{circle_id}"]').send_keys(Keys.ENTER)
        time.sleep(0.5)
    except Exception as e:
        return []
    #밑으로 내려주기 내려주는거 횟수 저장하기 
    for _ in range(((circle_num // 20 )) * 5):
        itemlist = driver.find_element(By.XPATH,'//*[@id="listContents1"]/div')
        driver.execute_script("arguments[0].scrollBy(0, document.body.scrollHeight)", itemlist)
        
        #일정한 시간에 클릭을 하니까 벤하는거 같다 
        time.sleep(0.5)
    
    time.sleep(0.5)
    #피터팬은 주소랑 위치 따고 위치는 클릭 안하게 주소는 따로 저장하고 피터팬은 request로 정보따오자// 부동산 114는 뭐야 시발
    piterpen = driver.page_source
    piterpen_soup = bs(piterpen, 'html.parser')
    piterpen_lst = []
    

    for i in range(1, circle_num + 1):
        if i in piterpen_lst: continue
        data = defaultdict(str)
        #위경도 추가, 마트, 학교 여부 추가
        data['x'] = where_x
        data['y'] = where_y
        data['마트'] = mart
        data['학교'] = school
        # number = random.uniform(0, 3)
        # time.sleep(number)
        #제목누르기 사진이 있는거와 없는거가 찾이가 난다 
        try:
            driver.find_element(By.XPATH,f'//*[@id="listContents1"]/div/div/div/div[{i}]/div/a/div[1]/span').click()
            #사진 없음
            data['사진'] = '0'
        except:
            driver.find_element(By.XPATH,f'//*[@id="listContents1"]/div/div/div/div[{i}]/div/a[2]/div[1]/span').click()
            data['사진'] = '1'

        time.sleep(0.5)
        estimate = driver.page_source
        estimate_soup = bs(estimate, 'html.parser')

        #피터팬이거나 중개소가 2곳인곳은 그냥 스킵
        try:
            detail_estimate = estimate_soup.find('div', class_ = 'detail_contents_inner')

            data['제목'] = detail_estimate.find('h4').text # 제목
            data['가격'] = detail_estimate.find('div', class_ ='info_article_price' ).text # 가격
            data['소제목'] = detail_estimate.find('div', class_ ='info_article_feature' ).text #소제목
            
            # 표 데이터 
            # 소재지가 없는 곳은 따로 좌표를 기준으로 동까지 추가해줘야겠다
            for i in detail_estimate.find('div', class_ = 'detail_box--summary').find_all('tr'):
                for title, content  in zip(i.find_all('th'), i.find_all('td')):
                    data[title.text] = content.text
        except:
            continue

        #데이터 저장
        lst.append(data)

    return lst

boodongsan_data = []
# user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
option = webdriver.ChromeOptions()
option.add_argument("--start-maximized")
# option.add_argument('--user-agent=' + user_agent)
driver = webdriver.Chrome(options = option)
what_num = 1

for x, y, school, mart in tqdm(zip(data['x'], data['y'], data['학교'],data['마트'])):
    
    driver.get(f'https://new.land.naver.com/rooms?ms={y},{x},19&a=APT:OPST:ABYG:OBYG:GM:OR:VL:DDDGG:JWJT:SGJT:HOJT&e=RETAIL&aa=SMALLSPCRENT')
    time.sleep(2)
    page = driver.page_source
    soup = bs(page, 'html.parser')
    circle = soup.find('div',class_ = 'map_area').select('div > div > a')
    print(f'좌표 : {y},{x}, 원 갯수 {len(circle)}', what_num)
    what_num += 1
    for i in circle:
        try:
            circle_id = i['id']
            circle_num = i.select_one('span.sale_number').text.split('개')[0]
            boodongsan_data.extend(circle_data_crawling(circle_id,int(circle_num), x,y,school,mart))
            print(len(boodongsan_data))
        except:
            break
        

boodongsan_data = pd.DataFrame(boodongsan_data)
#데이터 저장
boodongsan_data.to_csv('../final_data/new_boodongsan_data_utf.csv', index=False, encoding='utf-8')
boodongsan_data.head()

좌표 : 37.5674586107,126.9796562972, 원 갯수 4 463


464it [3:05:36,  3.43s/it]

좌표 : 37.5682552772,126.9784687595, 원 갯수 4 464


465it [3:05:38,  3.07s/it]

좌표 : 37.5678013999,126.9821577316, 원 갯수 4 465


...

489it [3:06:33,  2.28s/it]

좌표 : 37.5119608522,126.8851896675, 원 갯수 4 489


490it [3:06:35, 22.85s/it]

좌표 : 37.5742826049,127.0385255699, 원 갯수 4 490


,x,y,마트,학교,사진,제목,가격,소제목,소재지,매물특징,...,총세대수,용도지역,주구조,전세금 반환보증,재건축/재개발,준공인가일,사용검사일,대지/연면적,건축/전용면적,지상층/지하층
0,126.963506,37.47709,0,0,0,일반원룸6층,"전세2억 2,000(2,582만원/3.3㎡)",해당층/총층: 6/10층오픈형/단층 전용 면적:17.38㎡,서울시 관악구 봉천동 1660-12,"신축, 창이 넓고, 깔끔하고, 혼자 살기 좋은 방.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,126.963506,37.47709,0,0,0,일반원룸6층,"전세2억 2,000(2,582만원/3.3㎡)",해당층/총층: 6/10층오픈형/단층 전용 면적:17.38㎡,서울시 관악구 봉천동 1660-12,"신축, 창이 넓고, 깔끔하고, 혼자 살기 좋은 방.",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,126.963506,37.47709,0,0,0,동림 1동고층,"월세1,000/75",북향복도식 계약/전용 면적:42.35㎡/21.86㎡,NaN,복층오피스텔 입니다,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,126.963506,37.47709,0,0,0,동림 1동고층,"월세1,000/75",북향복도식 계약/전용 면적:42.35㎡/21.86㎡,NaN,복층오피스텔 입니다,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,126.963506,37.47709,0,0,0,오피스텔 o리더스빌o6층,"월세1,000/63",해당층/총층: 6/6층분리형/단층 전용 면적:20㎡,서울시 관악구 봉천동 1660-2,낙성대역 초역세권 대로변 오피스텔 1년계약,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
boodongsan_data = []
# user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
option = webdriver.ChromeOptions()
option.add_argument("--start-maximized")
# option.add_argument('--user-agent=' + user_agent)
driver = webdriver.Chrome(options = option)
driver.get(f'https://new.land.naver.com/rooms?ms=37.477090,126.963506,19&a=APT:OPST:ABYG:OBYG:GM:OR:VL:DDDGG:JWJT:SGJT:HOJT&e=RETAIL&aa=SMALLSPCRENT')





